In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import csv
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import pickle

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [3]:
from tensorflow.keras.models import Sequential #allows to build a sequential neural network
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf

In [4]:
from IPython.display import clear_output

In [5]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [6]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [7]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
   

In [8]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 


In [ ]:
#def extract_keypoints(results):
    #pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_world_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #return np.concatenate([pose])

In [9]:
def extract_keypoints2(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_world_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([pose])

In [10]:
final = np.load('dictionnairefinal.npy',allow_pickle=True).item()

In [11]:
classes = np.array(['drink water','brushing hair','pickup','throw','sitting down','standing up','clapping','cheer up',
           'hand waving','kicking','hopping','jump up','pointing to something','selfie','rub two hands','salute',
           'cross hands','sneeze/cough','falling','touch head','touch chest','touch back','touch neck'])

In [12]:
model = Sequential()
model.add(LSTM(64,return_sequences=True, activation='relu',input_shape=(30,99)))
model.add(LSTM(128,return_sequences=True, activation='relu'))
model.add(LSTM(256,return_sequences=True, activation='relu'))
model.add(LSTM(128,return_sequences=True, activation='relu'))
model.add(LSTM(64,return_sequences=False, activation='relu')) #3 couches of LSTM
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(classes.shape[0], activation='softmax'))

In [15]:
model.load_weights('reseau3.h5')

In [ ]:
def prediction(model,entry):
    entry = entry[:,:,:-1]#pour enlever la visibility
    seq = entry.reshape(30,33*3)
    res = model.predict(np.expand_dims(seq, axis=0))[0]
    a = np.argmax(res)
    b = max(res)
    return a,b

In [ ]:
a, b = prediction(model,final['A001'][18])
print(classes[a],b)

In [21]:
sequence = []
sentence = []
threshold = 0.995

#capture = r'C:\Users\Sohaib\OneDrive - UMONS\Documents\Projects\DL4HAR_StudentsProj\NTU-RGB\OneDrive_1_19-04-2022\S001C001P001R001A006_rgb.avi'
#cap = cv2.VideoCapture(capture)

capture = 0
cap = cv2.VideoCapture(capture, cv2.CAP_DSHOW)
cap.set(3, 1920)
cap.set(4, 1080)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        
        if ret:

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
    #         print(results)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints2(results)
    #         sequence.insert(0,keypoints)
    #         sequence = sequence[:30]
            sequence.append(keypoints)
            sequence = sequence[-30:]
            

            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
#                 print(classes[np.argmax(res)])


    #         #3. Viz logic
    #             if res[np.argmax(res)] > threshold: 
    #                 if len(sentence) > 0: 
    #                     if classes[np.argmax(res)] != sentence[-1]:
    #                         sentence.append(classes[np.argmax(res)])
    #                 else:
    #                     sentence.append(classes[np.argmax(res)])

    #             if len(sentence) > 5: 
    #                 sentence = sentence[-5:]

    #             # Viz probabilities
    #             image = prob_viz(res, classes, image, colors)
                print(np.max(res))
                if np.max(res) >= threshold:
                    cv2.putText(image, classes[np.argmax(res)] + ' - ' + str(np.max(res)), (3,30), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    #         cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)


            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

0.99620265
0.9886945
0.88740224
0.8215829
0.9625806
0.92030627
0.9976267
0.99456906
0.999944
0.9997911
0.86183405
0.88779604
0.99923325
0.9996418
0.99610776
0.9695087
0.86494136
0.7732419
0.6210643
0.7452726
0.8629661
0.9633514
0.91442114
0.59414905
0.806228
0.5090688
0.7773443
0.9249297
0.94079554
0.99993384
0.9999951
0.99978536
0.6650904
0.9440144
0.9537544
0.96748054
0.9945633
0.9999136
0.9999968
0.9999771
0.9987465
0.9970879
0.9709245
0.9823742
0.99574983
0.8218283
0.6436088
0.46706137
0.70472693
0.9999995
0.99999845
0.9999994
0.99999976
0.9999993
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999
0.99885297
0.99999905
0.99993765
0.9999976
0.99999726
0.9999894
0.99999595
0.99997413
0.9999895
0.99999964
0.9999875
0.9997707
0.99393725
0.6339256
0.8737546
0.98281693
0.98213905
0.9843902
0.98511654
0.9397547
0.7168089
0.4586073
0.8799329
0.96329665
0.8602693
0.54759187
0.9889353
0.9958248
0.6031495
0.764988
0.521316
0.8042957
0

0.9999553
0.9999856
0.99999726
0.99999774
0.99999714
0.9999968
0.9999968
0.9999964
0.9999962
0.99999547
0.9999944
0.9999927
0.9999913
0.99998915
0.9999869
0.9999844
0.99998
0.99997234
0.99995744
0.99993694
0.99990416
0.99981695
0.99965215
0.99936265
0.99897575
0.99864537
0.9985079
0.99843353
0.99840075
0.9985252
0.99872154
0.9989242
0.99918014
0.99942267
0.9994461
0.9991394
0.9980804
0.9916666
0.99529994
0.9993604
0.9999237
0.99993014
0.9988815
0.95014095
0.65974665
0.583067
0.7305833
0.8603406
0.9268694
0.95162535
0.96765196
0.9752983
0.9742431
0.9684507
0.9406261
0.935526
0.95161027
0.9498655
0.8330625
0.655702
0.65910256
0.4767485
0.7985236
0.9988109
0.99999917
0.9999999
0.99999976
0.9999994
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99999917
0.9999907
0.9999912
0.9999957
0.99999607
0.9999914
0.99996006
0.99907
0.9587005
0.8841987
0.5428814
0.7501514
0.9492869
0.9867703
0.96523786
0.95267713
0.93691224
0.97174495
0.94339657
0.7759851
0.77827257
0.8078346
0.6311766
0.478

0.999992
0.99999166
0.99999833
0.9999969
0.9998747
0.72104347
0.50875264
0.9819649
0.9798881
0.94906604
0.50469446
0.40046456
0.70458627
0.9997906
0.9999205
0.99996567
0.82089496
0.9986854
0.9997837
0.8578285
0.6924571
0.86676186
0.93423176
0.99216026
0.9979984
0.99995255
0.99998105
0.99803907
0.9939832
0.989223
0.9875683
0.99987197
0.9986375
0.99745744
0.99981016
0.99966824
0.9985656
0.8468094
0.984444
0.9993808
0.999944
0.9999864
0.999726
0.98999816
0.97837055
0.98204297
0.9589702
0.6298357
0.94513714
0.97339433
0.9658919
0.9782752
0.9881479
0.4725527
0.8643763
0.8046101
0.6805221
0.5025239
0.79832846
0.5461965
0.8812817
0.9043654
0.8833588
0.78581685
0.7827186
0.69747335
0.42907533
0.34590796
0.48254037
0.2900596
0.5127865
0.9225546
0.9956929
0.9995339
0.9995778
0.9992489
0.9982284
0.98955566
0.9292104
0.93042195
0.98888355
0.994055
0.9880304
0.98539925
0.6281779
0.51531446
0.5951771
0.9936335
0.5046932
0.7129374
0.87250423
0.9097621
0.6795804
0.4065896
0.54822576
0.9608727
0.998127

0.94518185
0.9692821
0.91450846
0.6650759
0.8820227
0.93776613
0.9976186
0.99865854
0.99381894
0.99993
0.9999709
0.99983203
0.9984907
0.9963146
0.98625076
0.98132724
0.6532643
0.97360367
0.99847156
0.9994791
0.95915055
0.52795935
0.95270604
0.9364816
0.9481281
0.96768135
0.985816
0.99143434
0.98992366
0.9854916
0.9845828
0.98560035
0.98563623
0.9854122
0.98358953
0.9829806
0.9831814
0.9855548
0.9843018
0.95667267
0.84665835
0.5257444
0.42932037
0.4875253
0.5756389
0.88938034
0.9445422
0.9458159
0.81521136
0.47213608
0.5649299
0.4495602
0.5459984
0.6786137
0.68249995
0.74374855
0.8138569
0.86402786
0.83774
0.73502725
0.5136568
0.90570986
0.9971853
0.99975055
0.999972
0.99999595
0.99999845
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99999976
1.0
1.0
1.0
1.0
1.0
0.9999994
0.99998164
0.9990127
0.9281896
0.6653703
0.50761116
0.8807506
0.99151844
0

In [19]:
    cap.release()
    cv2.destroyAllWindows()